In [4]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 12 11:49:08 2019
@author: wangxinyi
"""
import pymysql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import math

from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from dateutil import parser
from scipy import stats  
from scipy.stats import entropy
from sklearn.preprocessing import StandardScaler

conn= pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='root123',charset='utf8',db='car_evaluation')

all_data_df=pd.read_sql_query('''
select id,ty,substr(t,1,7) as t, max(x) as x,max(y) as y,ROUND(avg(v),2) as v,ROUND(avg(d),0) as d  
from tianchi_01_train  group by id,ty,substr(t,1,9)  order by id,t 
''',conn)


In [5]:
all_data_df.to_csv('data_train_o_02.csv')

In [14]:
all_df = all_data_df.pivot_table(values=['x', 'y','v','d'], index=['id', 'ty'],columns=['t'],fill_value=0.0)
all_df=all_df.reset_index()
all_df[['x','y','v','d']]

In [54]:
d = defaultdict(LabelEncoder)
df = all_df[['ty']].apply(lambda x:d[x.name].fit_transform(x) if type(x[0]) is str or math.isnan(x[0])  else x)

In [55]:
scaler = StandardScaler()
x_train= scaler.fit_transform(all_df[['x','y','v','d']])
#x_train = df.loc[:,names_x].values
y_train = df[['ty']].values

In [66]:
x_train.shape[1]

2256

In [2]:
import pandas as pd
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.5.final.0
python-bits: 64
OS: Windows
OS-release: 7
machine: AMD64
processor: Intel64 Family 6 Model 78 Stepping 3, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.23.0
pytest: 3.5.1
pip: 20.0.2
setuptools: 39.1.0
Cython: 0.28.2
numpy: 1.17.3
scipy: 1.4.1
pyarrow: None
xarray: None
IPython: 6.4.0
sphinx: 1.7.4
patsy: 0.5.0
dateutil: 2.7.3
pytz: 2018.4
blosc: None
bottleneck: 1.2.1
tables: 3.4.3
numexpr: 2.6.5
feather: None
matplotlib: 2.2.2
openpyxl: 2.5.3
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.0.4
lxml: 4.2.1
bs4: 4.6.0
html5lib: 0.9999999
sqlalchemy: 1.2.7
pymysql: 0.9.3
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None


In [69]:
import keras
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout, merge, Add
from keras.callbacks import Callback, EarlyStopping
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

def NN_model():
    init = keras.initializers.glorot_uniform(seed=1)
    model = keras.models.Sequential()
    model.add(Dense(units=1024, input_dim=x_train.shape[1], kernel_initializer=init, activation='relu'))
    model.add(Dense(units=512, kernel_initializer=init, activation='relu'))
    model.add(Dense(units=156, kernel_initializer=init, activation='relu'))
    model.add(Dense(units=128, kernel_initializer=init, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(units=64, kernel_initializer=init, activation='relu'))
    #model.add(Dense(units=48, kernel_initializer=init, activation='relu'))
    model.add(Dense(units=32, kernel_initializer=init, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=16, kernel_initializer=init, activation='softplus'))
    model.add(Dense(units=8, kernel_initializer=init, activation='softplus'))
    model.add(Dense(units=3, kernel_initializer=init, activation='softmax'))
    
    return model

class Metric(Callback):
    def __init__(self, model, callbacks, data):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.data = data

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, batch, logs=None):
        X_train, y_train = self.data[0][0], self.data[0][1]
        y_pred3 = self.model.predict(X_train)
        y_pred = np.zeros((len(y_pred3), ))
        y_true = np.zeros((len(y_pred3), ))
        for i in range(len(y_pred3)):
            y_pred[i] = list(y_pred3[i]).index(max(y_pred3[i]))
        for i in range(len(y_pred3)):
            y_true[i] = list(y_train[i]).index(max(y_train[i]))
        trn_s = f1_score(y_true, y_pred, average='macro')
        logs['trn_score'] = trn_s
        
        X_val, y_val = self.data[1][0], self.data[1][1]
        y_pred3 = self.model.predict(X_val)
        y_pred = np.zeros((len(y_pred3), ))
        y_true = np.zeros((len(y_pred3), ))
        for i in range(len(y_pred3)):
            y_pred[i] = list(y_pred3[i]).index(max(y_pred3[i]))
        for i in range(len(y_pred3)):
            y_true[i] = list(y_val[i]).index(max(y_val[i]))
        val_s = f1_score(y_true, y_pred, average='macro')
        logs['val_score'] = val_s
        print('trn_score', trn_s, 'val_score', val_s)

        for callback in self.callbacks:
            callback.on_epoch_end(batch, logs)

In [61]:
from keras.utils import to_categorical
y_train_c = to_categorical(y_train)
y_train_c[100:110]

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [70]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=20200203)


import keras
from sklearn import metrics

b_size = 2
max_epochs = 200
models = []
for fold, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train)):
    print('fold:', fold)
    X_train, Y_train = x_train[trn_idx], y_train_c[trn_idx] #df[names_x].loc[trn_idx], df[names_y].loc[trn_idx] # x_train[trn_idx], y_train[trn_idx]
    X_val, Y_val = x_train[val_idx], y_train_c[val_idx]
    Y_val_o=y_train[val_idx]
    
    model = NN_model()
    simple_adam = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer=simple_adam, metrics=['accuracy'])
    es = EarlyStopping(monitor='val_score', patience=50, verbose=1, mode='max')#, restore_best_weights=True,)
    es.set_model(model)
    metric = Metric(model, [es], [(X_train, Y_train), (X_val, Y_val)])
    model.fit(X_train, Y_train, batch_size=b_size, epochs=max_epochs, 
              validation_data = [X_val, Y_val],
              callbacks=[metric], shuffle=True, verbose=1)
    
    models.append(model)
    test_pred = model.predict(X_val)
    test_pred_v = np.argmax(test_pred, axis=1)
        

    score_ = metrics.f1_score(Y_val_o, test_pred_v, average='macro')
    print('*'*20)
    print('total score is : {}'.format(score_))
    print('*'*20)
    break


fold: 0
Train on 5600 samples, validate on 1400 samples
Epoch 1/200
5600/5600 [==============================] - 229s 41ms/step - loss: 1.0312 - acc: 0.5946 - val_loss: 0.8993 - val_acc: 0.6236
trn_score 0.255868544600939 val_score 0.25604927408710954
Epoch 2/200
1388/5600 [======>.......................] - ETA: 2:46 - loss: 0.8963 - acc: 0.6354

KeyboardInterrupt: 

In [104]:
score_ = metrics.f1_score(Y_val_o, test_pred_v, average='macro')
score_
#0.7418763267415228
#0.7319544696703284

0.7168176353499538

In [52]:
all_test_df = pd.read_sql_query('''select id,x,y,v,d,t,concat(x,'-',y)  as xy from tianchi_01_test  order by  id,t  ''',conn)


test_df=all_test_df.groupby(['id']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'y': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75] ,
                                              'd':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75],
                                              't':[t_p,month,day],
                                              'xy':[distinct_count,distance_median]
                                      })
test_df.columns=names_x_x
test_df=test_df.reset_index()

In [30]:
test_df

,id,c_x,min_x,max_x,median_x,model_x,c_y,min_y,max_y,median_y,...,model_v,d_v,avg_d,stdev_d,median_d,entropy_d,model_d,d_d,t,d_xy
0,7000,7.092492e+06,7.119130e+06,7.069441e+06,7.087014e+06,7.083957e+06,5.918416e+06,5.966537e+06,5.893360e+06,5.918174e+06,...,0.110,63.0,137.356568,115.674525,126.0,6.080001,0.0,147,6.091312e+06,160
1,7001,6.239019e+06,6.254992e+06,6.216428e+06,6.240004e+06,6.246625e+06,5.212498e+06,5.242126e+06,5.176732e+06,5.213941e+06,...,0.110,76.0,149.606987,104.616537,185.0,6.180437,40.0,144,1.380885e+05,367
2,7002,6.678498e+06,6.745702e+06,6.566008e+06,6.670449e+06,6.696588e+06,5.482646e+06,5.567766e+06,5.385267e+06,5.457670e+06,...,0.700,89.0,159.436585,113.566486,153.0,6.373595,50.0,163,1.540428e+05,396
3,7003,6.160467e+06,6.205663e+06,6.150675e+06,6.150875e+06,6.150875e+06,5.202435e+06,5.206247e+06,5.174729e+06,5.206031e+06,...,0.110,49.0,122.242353,119.624570,100.0,5.419250,0.0,147,2.228096e+05,71
4,7004,6.357542e+06,6.407177e+06,6.348864e+06,6.349065e+06,6.349065e+06,5.398000e+06,5.406460e+06,5.343990e+06,5.406460e+06,...,0.110,59.0,123.839196,116.631778,116.0,5.805988,0.0,158,2.379218e+05,100
5,7005,6.322045e+06,6.322055e+06,6.321954e+06,6.322055e+06,6.322055e+06,5.381925e+06,5.381943e+06,5.381833e+06,5.381942e+06,...,0.000,13.0,80.019802,109.330694,0.0,4.097435,0.0,112,1.565428e+05,4
6,7006,7.054221e+06,7.067504e+06,7.051061e+06,7.054275e+06,7.052163e+06,5.746713e+06,5.783325e+06,5.734358e+06,5.743727e+06,...,2.725,49.0,148.895570,98.147307,172.5,6.783413,178.0,141,2.996635e+05,286
7,7007,6.073201e+06,6.102651e+06,6.022657e+06,6.073013e+06,6.102649e+06,5.053933e+06,5.112762e+06,5.002226e+06,5.037612e+06,...,0.000,63.0,105.063260,106.541190,72.0,5.738276,0.0,163,5.528127e+06,280
8,7008,6.325893e+06,6.336889e+06,6.325721e+06,6.325721e+06,6.325721e+06,5.286243e+06,5.286323e+06,5.284157e+06,5.286256e+06,...,0.160,22.0,103.948837,111.857582,76.0,5.541970,0.0,187,1.468784e+05,10
9,7009,6.572146e+06,6.574011e+06,6.564846e+06,6.573241e+06,6.573241e+06,5.479199e+06,5.501269e+06,5.455010e+06,5.481878e+06,...,0.650,41.0,150.479714,103.255786,155.0,7.573890,0.0,240,1.507346e+05,148


In [75]:
# x_test = scaler.fit_transform(test_df[names_x])
x_test = test_df.loc[:,names_x].values

test_y_p= search.predict_proba(x_test)

In [54]:
x_test = test_df.loc[:,names_x].values
test_y= search.predict(x_test)
test_df['p_ty']=test_y
test_df['p_ty_str']=d['ty'].inverse_transform(test_y)
test_df['proba_0']=test_y_p[:,0]
test_df['proba_1']=test_y_p[:,1]
test_df['proba_2']=test_y_p[:,2]

In [55]:
test_df[['id','p_ty_str','p_ty','proba_0','proba_1','proba_2']]

,id,p_ty_str,p_ty,proba_0,proba_1,proba_2
0,7000,围网,1,0.029375,0.922732,0.047894
1,7001,拖网,2,0.044678,0.037702,0.917620
2,7002,围网,1,0.029657,0.885140,0.085203
3,7003,拖网,2,0.040819,0.037593,0.921587
4,7004,围网,1,0.029524,0.900886,0.069590
5,7005,拖网,2,0.449702,0.038231,0.512066
6,7006,围网,1,0.115559,0.825555,0.058886
7,7007,拖网,2,0.029209,0.037491,0.933300
8,7008,围网,1,0.047377,0.904069,0.048554
9,7009,拖网,2,0.030322,0.257067,0.712612


In [57]:
result_df=test_df[['id','p_ty_str']]
result_df.to_csv("tianchi01_result_0202_knn.csv",index=False,header=0)

In [56]:
result_df_p=test_df[['id','p_ty_str','p_ty','proba_0','proba_1','proba_2']]
result_df_p['proba_max']=result_df_p[['proba_0','proba_1','proba_2']].max(axis=1)
result_df_p=result_df_p[['id','p_ty_str','proba_max']]
result_df_p.to_csv("tianchi01_result_p_0202_knn.csv",index=False,header=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
